# PyWarlight RL Agent Setup and Execution

This notebook provides a complete setup and execution environment for running the PyWarlight reinforcement learning agents. It will install all dependencies and run a simulation with the specified configuration.

## Overview
- **Option 1**: Clone the project directly from GitHub (recommended for Colab)
- **Option 2**: Use local files (for local Jupyter environments)
- Installs all required Python packages from requirements.txt
- Sets up the environment for running RL agent simulations
- Executes the command: `python main.py Attila.1 RLGNNAgent.2 -sim 1 -map World -config debug`

## 0. Setup Project Files

Choose your setup method:
- **Method A**: Clone from GitHub (recommended for Google Colab)
- **Method B**: Use local files (for local Jupyter environments)

In [ ]:
import os
import sys
import subprocess

# Configuration
USE_GITHUB = True  # Set to False if you want to use local files instead
GITHUB_USERNAME = "DonnyWhoLovedBowling"
REPO_NAME = "pyWarlight"
GITHUB_URL = f"https://github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"

print("🚀 PyWarlight RL Agent Setup")
print("=" * 50)

if USE_GITHUB:
    print(f"📂 Method A: Cloning from GitHub repository")
    print(f"Repository: {GITHUB_URL}")
    
    # Check if git is available
    try:
        subprocess.run(['git', '--version'], check=True, capture_output=True)
        print("✅ Git is available")
    except (subprocess.CalledProcessError, FileNotFoundError):
        print("❌ Git not found. Installing git...")
        # For Google Colab
        try:
            subprocess.run(['apt-get', 'update', '-qq'], check=True)
            subprocess.run(['apt-get', 'install', '-y', 'git'], check=True)
            print("✅ Git installed successfully")
        except:
            print("❌ Could not install git. Please install manually or use local files.")
            USE_GITHUB = False
    
    if USE_GITHUB:
        # Clone the repository
        if os.path.exists(REPO_NAME):
            print(f"📁 Directory {REPO_NAME} already exists. Updating...")
            os.chdir(REPO_NAME)
            result = subprocess.run(['git', 'pull'], capture_output=True, text=True)
            if result.returncode == 0:
                print("✅ Repository updated successfully")
            else:
                print(f"⚠️ Update failed: {result.stderr}")
                print("Continuing with existing files...")
        else:
            print(f"📥 Cloning repository...")
            result = subprocess.run(['git', 'clone', GITHUB_URL], capture_output=True, text=True)
            if result.returncode == 0:
                print("✅ Repository cloned successfully")
                os.chdir(REPO_NAME)
            else:
                print(f"❌ Clone failed: {result.stderr}")
                print("Falling back to local files method...")
                USE_GITHUB = False

if not USE_GITHUB:
    print(f"📂 Method B: Using local files")
    print("Make sure you have the pyWarlight project files in the current directory.")
    
    # For Google Colab, provide file upload option
    try:
        from google.colab import files
        import zipfile
        
        print("\n📤 Google Colab detected. You can upload a ZIP file of your project:")
        print("1. Create a ZIP file of your pyWarlight project")
        print("2. Run the next cell to upload it")
        print("\nAlternatively, you can skip upload if files are already present.")
        
        # Check if we should prompt for upload
        if not os.path.exists('main.py'):
            print("\n⬆️ Upload your project ZIP file:")
            uploaded = files.upload()
            
            # Extract ZIP file
            for filename in uploaded.keys():
                if filename.endswith('.zip'):
                    print(f"📦 Extracting {filename}...")
                    with zipfile.ZipFile(filename, 'r') as zip_ref:
                        zip_ref.extractall('.')
                    
                    # Check if extracted into a subdirectory
                    if os.path.exists('pyWarlight'):
                        os.chdir('pyWarlight')
                        print("📁 Moved to pyWarlight directory")
                    
                    print("✅ Files extracted successfully")
                    break
        else:
            print("✅ Project files found in current directory")
            
    except ImportError:
        # Not in Colab
        print("💻 Local Jupyter environment detected")
        if not os.path.exists('main.py'):
            print("❌ main.py not found in current directory")
            print("Please ensure you're running this notebook from the pyWarlight project directory.")

print(f"\n📍 Current working directory: {os.getcwd()}")
print(f"📄 Files in directory: {[f for f in os.listdir('.') if os.path.isfile(f)][:10]}")
print(f"📁 Subdirectories: {[d for d in os.listdir('.') if os.path.isdir(d)][:10]}")

## 1. Install System Requirements

Check the system environment and install any necessary system-level dependencies.

In [ ]:
import sys
import platform

print("🖥️ System Information:")
print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"Current working directory: {os.getcwd()}")
print(f"Python executable: {sys.executable}")

# Check if we're in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Running in Google Colab")
    
    # Enable GPU if available
    import torch
    if torch.cuda.is_available():
        print(f"🚀 CUDA available: {torch.cuda.get_device_name(0)}")
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    else:
        print("💻 CUDA not available, using CPU")
        
except ImportError:
    IN_COLAB = False
    print("💻 Running in local environment")

# Install system dependencies for Colab
if IN_COLAB:
    print("\n📦 Installing system dependencies for Colab...")
    try:
        subprocess.run(['apt-get', 'update', '-qq'], check=True)
        subprocess.run(['apt-get', 'install', '-y', 'build-essential'], check=True)
        print("✅ System dependencies installed")
    except Exception as e:
        print(f"⚠️ System dependency installation failed: {e}")

## 2. Install Python Dependencies

Install all required Python packages, with optimizations for different environments.

In [ ]:
print("📦 Installing Python dependencies...")

# For Google Colab, install PyTorch with CUDA support first
if IN_COLAB:
    print("🔥 Installing PyTorch with CUDA support for Colab...")
    
    # Install PyTorch with CUDA
    torch_result = subprocess.run([
        sys.executable, '-m', 'pip', 'install', 
        'torch', 'torchvision', 'torchaudio', 
        '--index-url', 'https://download.pytorch.org/whl/cu118'
    ], capture_output=True, text=True)
    
    if torch_result.returncode == 0:
        print("✅ PyTorch with CUDA installed successfully")
    else:
        print(f"⚠️ PyTorch installation warning: {torch_result.stderr}")
    
    # Install PyTorch Geometric with CUDA support
    print("📊 Installing PyTorch Geometric with CUDA support...")
    pyg_packages = [
        'torch-geometric',
        'pyg-lib', 'torch-scatter', 'torch-sparse', 
        'torch-cluster', 'torch-spline-conv'
    ]
    
    for package in pyg_packages:
        pyg_result = subprocess.run([
            sys.executable, '-m', 'pip', 'install', package
        ], capture_output=True, text=True)
        
        if pyg_result.returncode == 0:
            print(f"✅ {package} installed")
        else:
            print(f"⚠️ {package} installation issue: {pyg_result.stderr[:100]}...")

# Install from requirements.txt if available
if os.path.exists('requirements.txt'):
    print("\n📋 Found requirements.txt. Installing dependencies...")
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'],
        capture_output=True,
        text=True
    )
    print("STDOUT:")
    print(result.stdout[-1000:])  # Show last 1000 chars
    if result.stderr:
        print("STDERR:")
        print(result.stderr[-1000:])  # Show last 1000 chars
    print(f"Installation completed with return code: {result.returncode}")
else:
    print("\n📦 requirements.txt not found. Installing common dependencies...")
    essential_packages = [
        'torch',
        'torch_geometric', 
        'tensorboard', 
        'multipledispatch', 
        'statsmodels',
        'matplotlib',
        'numpy',
        'pandas'
    ]
    
    for package in essential_packages:
        print(f"Installing {package}...")
        result = subprocess.run(
            [sys.executable, '-m', 'pip', 'install', package],
            capture_output=True,
            text=True
        )
        if result.returncode == 0:
            print(f"✅ {package} installed successfully")
        else:
            print(f"❌ {package} installation failed: {result.stderr[:100]}...")

print("\n🎉 Dependency installation completed!")

## 3. Verify Installation

Check that all critical dependencies are properly installed and accessible.

In [ ]:
# Check critical imports
required_packages = {
    'torch': 'PyTorch for deep learning',
    'torch_geometric': 'PyTorch Geometric for graph neural networks',
    'tensorboard': 'TensorBoard for logging',
    'multipledispatch': 'Multiple dispatch for Python',
    'statsmodels': 'Statistical models',
    'numpy': 'Numerical computing',
    'matplotlib': 'Plotting library'
}

print("🔍 Verifying package installations:\n")
for package, description in required_packages.items():
    try:
        module = __import__(package)
        # Try to get version if available
        version = getattr(module, '__version__', 'unknown')
        print(f"✅ {package:<20} v{version:<10} - {description}")
    except ImportError as e:
        print(f"❌ {package:<20} {'FAILED':<10} - {e}")

# Check PyTorch CUDA availability
try:
    import torch
    if torch.cuda.is_available():
        device_name = torch.cuda.get_device_name(0)
        print(f"\n🚀 CUDA Status: Available ({device_name})")
        print(f"   CUDA Version: {torch.version.cuda}")
        print(f"   PyTorch Version: {torch.__version__}")
    else:
        print(f"\n💻 CUDA Status: Not available (using CPU)")
        print(f"   PyTorch Version: {torch.__version__}")
except ImportError:
    print("\n❌ PyTorch not available")

# Check if main project files exist
print(f"\n📁 Checking project files:")
essential_files = ['main.py', 'requirements.txt', 'src/']
for file in essential_files:
    if os.path.exists(file):
        if os.path.isfile(file):
            size = os.path.getsize(file)
            print(f"✅ {file:<15} ({size:,} bytes)")
        else:
            items = len(os.listdir(file)) if os.path.isdir(file) else 0
            print(f"✅ {file:<15} ({items} items)")
    else:
        print(f"❌ {file:<15} (not found)")

# List current directory contents
print(f"\n📋 Current directory contents:")
items = sorted(os.listdir('.'))
for item in items[:15]:  # Show first 15 items
    if os.path.isfile(item):
        size = os.path.getsize(item)
        print(f"📄 {item:<25} ({size:,} bytes)")
    else:
        try:
            count = len(os.listdir(item))
            print(f"📁 {item}/<20} ({count} items)")
        except:
            print(f"📁 {item}/")

if len(items) > 15:
    print(f"... and {len(items) - 15} more items")

print("\n🎯 Installation verification completed!")

## 4. Set Up Environment Variables

Configure any necessary environment variables or path settings for the project.

In [ ]:
# Set up environment variables
print("⚙️ Setting up environment variables...")

# Environment variables for optimal performance
env_vars = {
    'PYTHONPATH': os.getcwd(),  # Add current directory to Python path
    'TF_ENABLE_ONEDNN_OPTS': '0',  # Disable oneDNN for consistent results
    'CUDA_LAUNCH_BLOCKING': '1',  # For better CUDA error reporting
    'TORCH_USE_CUDA_DSA': '1',  # Enhanced CUDA debugging
}

# Add GPU-specific settings if available
try:
    import torch
    if torch.cuda.is_available() and IN_COLAB:
        env_vars['CUDA_VISIBLE_DEVICES'] = '0'
        print("🚀 GPU-specific environment variables added")
except:
    pass

for var, value in env_vars.items():
    os.environ[var] = value
    print(f"✅ Set {var} = {value}")

print("\n📍 Environment setup complete!")
print(f"Current PYTHONPATH: {os.environ.get('PYTHONPATH', 'Not set')}")
print(f"Working directory: {os.getcwd()}")

# Verify Python can find the project modules
sys.path.insert(0, os.getcwd())
print(f"✅ Added current directory to Python path")

## 5. Execute Main Command

Run the main command with the specified configuration: `python main.py Attila.1 RLGNNAgent.2 -sim 1 -map World -config debug`

In [ ]:
# Prepare the command
command = [
    sys.executable,  # Use the same Python interpreter as the notebook
    'main.py',
    'Attila.1',
    'RLGNNAgent.2',
    '-sim', '1',
    '-map', 'World',
    '-config', 'debug'
]

print(f"🎮 Executing command: {' '.join(command)}")
print(f"📍 Working directory: {os.getcwd()}")
print(f"🐍 Python executable: {sys.executable}")
print("=" * 80)

# Verify main.py exists before running
if not os.path.exists('main.py'):
    print("❌ Error: main.py not found in current directory!")
    print(f"Current directory: {os.getcwd()}")
    print(f"Files available: {[f for f in os.listdir('.') if f.endswith('.py')]}")
    print("Please ensure the project files are properly loaded.")
else:
    # Execute the command
    try:
        # Run the command and capture output
        process = subprocess.Popen(
            command,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
            universal_newlines=True,
            cwd=os.getcwd()  # Explicitly set working directory
        )
        
        # Stream output in real-time
        output_lines = []
        for line in process.stdout:
            line_clean = line.rstrip()
            print(line_clean)
            output_lines.append(line_clean)
        
        # Wait for process to complete
        return_code = process.wait()
        
        print("=" * 80)
        print(f"🏁 Command completed with return code: {return_code}")
        
        # Store output for later analysis
        full_output = '\n'.join(output_lines)
        
        if return_code == 0:
            print("✅ Simulation completed successfully!")
        else:
            print("❌ Simulation failed - check output above for errors")
        
    except Exception as e:
        print(f"❌ Error executing command: {e}")
        return_code = -1
        full_output = ""

## 6. Capture and Display Output

Analyze the command output and display key information about the simulation results.

In [ ]:
# Analyze the output
print("📊 === EXECUTION SUMMARY ===")
print(f"Return Code: {return_code}")
print(f"Execution Status: {'✅ SUCCESS' if return_code == 0 else '❌ FAILED'}")

if 'full_output' in locals() and full_output:
    # Extract key information from the output
    lines = full_output.split('\n')
    
    print(f"\n📈 === OUTPUT ANALYSIS ===")
    print(f"Total output lines: {len(lines)}")
    
    # Look for configuration messages
    config_lines = [line for line in lines if any(word in line.lower() 
                   for word in ['configuration', 'config', 'applied', 'switching'])]
    if config_lines:
        print("\n📝 Configuration Messages:")
        for line in config_lines[:5]:  # Show first 5 config lines
            print(f"  {line}")
    
    # Look for training/model information
    training_lines = [line for line in lines if any(word in line.lower() 
                     for word in ['training', 'model', 'epoch', 'loss', 'learning'])]
    if training_lines:
        print("\n🏗️ Model/Training Information:")
        for line in training_lines[:5]:  # Show first 5 training lines
            print(f"  {line}")
    
    # Look for game results
    result_lines = [line for line in lines if any(word in line.lower() 
                   for word in ['won', 'result', 'winner', 'game', 'finished'])]
    if result_lines:
        print("\n🎮 Game Results:")
        for line in result_lines[-5:]:  # Show last 5 result lines
            print(f"  {line}")
    
    # Look for performance metrics
    perf_lines = [line for line in lines if any(word in line.lower() 
                 for word in ['fps', 'time', 'duration', 'speed', 'performance'])]
    if perf_lines:
        print("\n⚡ Performance Metrics:")
        for line in perf_lines[-3:]:  # Show last 3 performance lines
            print(f"  {line}")
    
    # Look for errors or warnings (show only serious ones)
    error_lines = [line for line in lines if any(word in line.lower() 
                  for word in ['error', 'exception', 'failed', 'critical'])]
    warning_lines = [line for line in lines if 'warning' in line.lower()]
    
    if error_lines:
        print("\n❌ Errors:")
        for line in error_lines[:5]:  # Show first 5 error lines
            print(f"  {line}")
    
    if warning_lines and len(warning_lines) <= 10:  # Only show if not too many
        print("\n⚠️ Warnings:")
        for line in warning_lines[:5]:  # Show first 5 warning lines
            print(f"  {line}")
    elif len(warning_lines) > 10:
        print(f"\n⚠️ Warnings: {len(warning_lines)} warnings detected (suppressed for brevity)")
    
    # Summary statistics
    error_count = len(error_lines)
    warning_count = len(warning_lines)
    print(f"\n📊 Summary: {error_count} errors, {warning_count} warnings")
    
else:
    print("\n❌ No output captured.")

print(f"\n🎯 === EXECUTION COMPLETE ===")
print("The PyWarlight RL simulation has finished running.")

if return_code == 0:
    print("\n✅ SUCCESS: The simulation completed successfully!")
    print("🎉 You can now analyze the results or run additional simulations.")
    if IN_COLAB:
        print("💡 Tip: Use TensorBoard to visualize training progress if logs were generated.")
else:
    print("\n❌ FAILED: The simulation encountered issues.")
    print("🔍 Check the output above for error messages and troubleshooting information.")
    print("\n🛠️ Common fixes:")
    print("   - Ensure all dependencies are properly installed")
    print("   - Check that project files were loaded correctly")
    print("   - Verify the configuration parameters are valid")

print(f"\n📁 Final working directory: {os.getcwd()}")
print(f"📄 Generated files: {[f for f in os.listdir('.') if f.endswith(('.log', '.pt', '.pkl', '.json'))]}")